In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer

In [11]:
df_train = pd.read_csv('application_train.csv')

In [12]:
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_test = pd.read_csv('application_test.csv')

### Run this cell for mem checks

In [14]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_train', 570142137),
 ('X', 565221961),
 ('df_test', 90020943),
 ('X_test', 89630991),
 ('y', 2460192),
 ('GaussianNB', 1056),
 ('GradientBoostingClassifier', 1056),
 ('Imputer', 1056),
 ('KFold', 1056),
 ('LogisticRegression', 1056),
 ('LogisticRegressionCV', 1056),
 ('RandomForestClassifier', 1056),
 ('SimpleImputer', 1056),
 ('StandardScaler', 1056),
 ('StratifiedKFold', 1056),
 ('accuracy_score', 136),
 ('confusion_matrix', 136),
 ('cross_val_score', 136),
 ('make_pipeline', 136),
 ('precision_score', 136),
 ('recall_score', 136),
 ('roc_auc_score', 136),
 ('train_test_split', 136),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80)]

In [15]:
X = df_train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = df_train['TARGET']

In [16]:
X_test = df_test.drop(['SK_ID_CURR'], axis=1)

### Find columns with nulls

In [17]:
columns_null = X.columns[X.count() != 307511]
columns_null

Index(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
       'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
       'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
       'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
       'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMI

In [18]:
X[columns_null].count()/307511

AMT_ANNUITY                     0.999961
AMT_GOODS_PRICE                 0.999096
NAME_TYPE_SUITE                 0.995799
OWN_CAR_AGE                     0.340092
OCCUPATION_TYPE                 0.686545
CNT_FAM_MEMBERS                 0.999993
EXT_SOURCE_1                    0.436189
EXT_SOURCE_2                    0.997854
EXT_SOURCE_3                    0.801747
APARTMENTS_AVG                  0.492503
BASEMENTAREA_AVG                0.414840
YEARS_BEGINEXPLUATATION_AVG     0.512190
YEARS_BUILD_AVG                 0.335022
COMMONAREA_AVG                  0.301277
ELEVATORS_AVG                   0.467040
ENTRANCES_AVG                   0.496512
FLOORSMAX_AVG                   0.502392
FLOORSMIN_AVG                   0.321514
LANDAREA_AVG                    0.406233
LIVINGAPARTMENTS_AVG            0.316450
LIVINGAREA_AVG                  0.498067
NONLIVINGAPARTMENTS_AVG         0.305670
NONLIVINGAREA_AVG               0.448208
APARTMENTS_MODE                 0.492503
BASEMENTAREA_MOD

### Inspect number of values in each categorical col

Some categorical columns are incorrectly coded as int64.  Convert them to objects.

In [19]:
df_train.FLAG_MOBIL.value_counts()

1    307510
0         1
Name: FLAG_MOBIL, dtype: int64

In [20]:
y.mean()

0.08072881945686496

In [21]:
#examine very lopsided cols to see if we can just remove them
import re
for col in df_train.columns:
    if re.search(r'FLAG', col):
        if df_train[col].value_counts().min()/df_train[col].value_counts().max() < .001:
            min_msk = df_train[col] == df_train[col].value_counts().argmin()
            print(col, df_train[col].value_counts().min(), y[min_msk].mean())

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  


FLAG_MOBIL 1 0.0
FLAG_DOCUMENT_2 13 0.3076923076923077
FLAG_DOCUMENT_4 25 0.0
FLAG_DOCUMENT_7 59 0.05084745762711865
FLAG_DOCUMENT_10 7 0.0
FLAG_DOCUMENT_12 2 0.0
FLAG_DOCUMENT_17 82 0.024390243902439025
FLAG_DOCUMENT_19 183 0.06557377049180328
FLAG_DOCUMENT_20 156 0.08333333333333333
FLAG_DOCUMENT_21 103 0.13592233009708737


flag doc 2, 21, 82, and 4(maybe) look important

In [22]:
#drop columns that don't have much info
X.drop(columns=['FLAG_MOBIL', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12'], inplace=True, errors='ignore')
X_test.drop(columns=['FLAG_MOBIL', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12'], inplace=True, errors='ignore')

### Visualize

In [46]:
# for col in df_train.columns[2:]:
#     try:
#         g = sns.FacetGrid(df_train[[col, 'TARGET']], hue='TARGET')
#         g = g.map(plt.hist, col)
#     except:
#         pass

In [47]:
y[X.DAYS_EMPLOYED > 0].mean()

0.05399646043269404

seems like it has an effect on target, so let's replace this with NaN and create new col that flags any weird number in this col

In [48]:
X['DAYS_EMPLOYED_ABNORMAL'] = (X['DAYS_EMPLOYED'] > 0).astype(int)
X_test['DAYS_EMPLOYED_ABNORMAL'] = (X_test['DAYS_EMPLOYED'] > 0).astype(int)

In [49]:
X['DAYS_EMPLOYED'][X['DAYS_EMPLOYED'] > 0] = np.nan
X_test['DAYS_EMPLOYED'][X_test['DAYS_EMPLOYED'] > 0] = np.nan

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
X['DAYS_EMPLOYED'].isnull().sum(), X_test['DAYS_EMPLOYED'].isnull().sum()

(55374, 9274)

### Imputation

In [51]:
# X_imputed = X.fillna(X.mean(numeric_only=True))

In [52]:
# X_imputed = X_imputed.fillna(X_imputed.mode().iloc[0])

In [53]:
#check to see if any null values in entire DF
# X_imputed.isnull().sum().sum()

In [54]:
# X_imputed['REGION_RATING_CLIENT'] = X_imputed['REGION_RATING_CLIENT'].astype(object)
# X_imputed['REGION_RATING_CLIENT_W_CITY'] = X_imputed['REGION_RATING_CLIENT_W_CITY'].astype(object)

In [55]:
# X_imputed = pd.get_dummies(X_imputed)

In [56]:
X['REGION_RATING_CLIENT'] = X['REGION_RATING_CLIENT'].astype(object)
X['REGION_RATING_CLIENT_W_CITY'] = X['REGION_RATING_CLIENT_W_CITY'].astype(object)
X_test['REGION_RATING_CLIENT'] = X_test['REGION_RATING_CLIENT'].astype(object)
X_test['REGION_RATING_CLIENT_W_CITY'] = X_test['REGION_RATING_CLIENT_W_CITY'].astype(object)

In [57]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

### Feature Engineering

In [103]:
X['CONSUMER'] = (X['AMT_CREDIT'] == X['AMT_GOODS_PRICE']).astype(int)
X_test['CONSUMER'] = (X_test['AMT_CREDIT'] == X_test['AMT_GOODS_PRICE']).astype(int)

In [108]:
X['TERM'] = X['AMT_CREDIT'] / X['AMT_ANNUITY']
X_test['TERM'] = X_test['AMT_CREDIT'] / X_test['AMT_ANNUITY']

In [109]:
X['ANNUITY_PER_INCOME'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
X_test['ANNUITY_PER_INCOME'] = X_test['AMT_ANNUITY'] / X_test['AMT_INCOME_TOTAL']

In [110]:
corrs = pd.concat([X, y],axis=1).corr()['TARGET'].sort_values()
corrs

EXT_SOURCE_3                                        -0.178919
EXT_SOURCE_2                                        -0.160472
EXT_SOURCE_1                                        -0.155317
NAME_EDUCATION_TYPE_Higher education                -0.056593
CODE_GENDER_F                                       -0.054704
NAME_INCOME_TYPE_Pensioner                          -0.046209
DAYS_EMPLOYED_ABNORMAL                              -0.045987
ORGANIZATION_TYPE_XNA                               -0.045987
FLOORSMAX_AVG                                       -0.044003
FLOORSMAX_MEDI                                      -0.043768
FLOORSMAX_MODE                                      -0.043226
EMERGENCYSTATE_MODE_No                              -0.042201
REGION_RATING_CLIENT_W_CITY_1                       -0.041945
REGION_RATING_CLIENT_1                              -0.040830
HOUSETYPE_MODE_block of flats                       -0.040594
AMT_GOODS_PRICE                                     -0.039645
REGION_P

In [111]:
corrs = abs(corrs).sort_values(ascending=False)
corrs = corrs.drop(index='TARGET', errors='ignore')

In [27]:
# X, X_test = X.align(X_test, axis=1, join='inner')

In [28]:
# X.shape, X_test.shape

((307511, 243), (48744, 243))